In [1]:
from AOC_utils import get_day
import numpy as np

day = 14
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Downloaded day 14 input
500


['p=67,43 v=80,86',
 'p=47,24 v=-9,-96',
 'p=6,38 v=10,81',
 'p=73,92 v=36,-93',
 'p=74,10 v=-35,19',
 'p=57,59 v=-5,71',
 'p=21,10 v=48,80']

In [2]:
# part 1
example = '''
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
'''.split('\n')[1:-1]

In [44]:
def solve_day(data, roomscale=[103, 101], time=100):
    part1 = 0
    part2 = 0

    robots = []
    for line in data:
        p, v = line.split(' ')
        p = np.array(p[2:].split(',')).astype(int)
        v = np.array(v[2:].split(',')).astype(int)
        robots.append([p, v])

    robots = np.array(robots)
    robots_pos = robots[:, 0]
    robots_vel = robots[:, 1]

    robots_vel *= time

    robots_pos += robots_vel
    robots_pos = robots_pos % roomscale

    # print(robots_pos)

    # print(robots_pos.shape)

    robots_topleft = robots_pos[np.bitwise_and(robots_pos[:,0] < roomscale[0]//2,
                                                  robots_pos[:,1] < roomscale[1]//2)]
    robots_topright = robots_pos[np.bitwise_and(robots_pos[:,0] < roomscale[0]//2,
                                                   robots_pos[:,1] > roomscale[1]//2)]
    robots_bottomleft = robots_pos[np.bitwise_and(robots_pos[:,0] > roomscale[0]//2,
                                                     robots_pos[:,1] < roomscale[1]//2)]
    robots_bottomright = robots_pos[np.bitwise_and(robots_pos[:,0] > roomscale[0]//2,
                                                        robots_pos[:,1] > roomscale[1]//2)]

    safety = robots_topleft.shape[0] * robots_topright.shape[0] * robots_bottomleft.shape[0] * robots_bottomright.shape[0]

    # print(robots_bottomleft.shape, robots_bottomright.shape, robots_topleft.shape, robots_topright.shape)

    part1 = safety

    room = np.full(np.array(roomscale), 0)
    room[robots_pos[:,0], robots_pos[:,1]] = 1

    structure = np.max(np.sum(room, axis=1)) + np.max(np.sum(room, axis=0))

    room = np.where(room == 1, '#', '.')

    if structure > 60:
        print()
        print(time)
        print(structure)
        [print(x) for x in room.tolist()]

    # print("part 1:", part1)
    # print("part 2:", part2)

solve_day(example, roomscale=[7, 11])

In [45]:
for i in range(10000):
    solve_day(input_data, roomscale=[101, 103], time=i)


7083
68
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',

In [24]:
solve_day(input_data, roomscale=[101, 103], time=100)

[[ 88  94]
 [ 56   3]
 [ 97   1]
 [ 37  62]
 [  8  56]
 [ 62  52]
 [ 74  79]
 [ 85  69]
 [ 81  11]
 [ 49  72]
 [ 86  14]
 [ 47  20]
 [ 91  60]
 [ 17  51]
 [ 17  47]
 [ 30  25]
 [  2  56]
 [  4  54]
 [ 24  88]
 [ 26  53]
 [  6  15]
 [ 15  49]
 [ 72  11]
 [ 74  49]
 [ 72   0]
 [  4  50]
 [ 48  91]
 [  0  38]
 [ 78  75]
 [ 15  83]
 [  2  57]
 [ 34  11]
 [ 16   2]
 [  8  71]
 [ 54  89]
 [ 24  61]
 [ 46  67]
 [ 55  23]
 [  4  10]
 [ 76 100]
 [ 66  25]
 [ 96  56]
 [ 27   2]
 [ 58  43]
 [ 96  41]
 [ 29  71]
 [ 51  39]
 [ 60  83]
 [ 69  66]
 [ 60  20]
 [ 88  78]
 [ 97  93]
 [ 62  69]
 [ 26  94]
 [ 29  71]
 [ 59  16]
 [ 28  61]
 [ 92   3]
 [  9  73]
 [ 16  98]
 [ 94  41]
 [  3  79]
 [ 31  95]
 [ 64  14]
 [ 34   8]
 [ 54  93]
 [ 80  51]
 [ 77  84]
 [ 23  69]
 [ 13  71]
 [ 40  31]
 [ 23  43]
 [ 52  99]
 [ 51  81]
 [ 68   2]
 [ 57  57]
 [ 23  10]
 [  9  18]
 [ 93  46]
 [ 69  85]
 [  9  54]
 [ 50  26]
 [ 61  88]
 [ 11  49]
 [ 87  54]
 [ 93  63]
 [ 35  70]
 [  0  51]
 [ 88  52]
 [  1  83]
 [ 64  17]

In [ ]:
221356422

In [ ]:
# part 2
example = '''

'''.split('\n')[1:-1]